# Fine-tuning and Optimizing a Student Model

This notebook demonstrates how to fine-tune a small "student" model using the training data we generated from our "teacher" model in the previous notebook. We'll also optimize the model for efficient deployment.

![](../../lab_manual/images/step-2.png)

## What You'll Learn

- How to use Microsoft Olive to fine-tune the Phi-4-mini model
- How to apply Low-Rank Adaptation (LoRA) for efficient fine-tuning
- How to convert a model to ONNX format for optimization
- How to apply quantization to reduce model size
- How to prepare the model for deployment on resource-constrained environments

## Prerequisites

- Completed the previous notebook (`01.AzureML_Distillation.ipynb`)
- Generated training data in `data/train_data.jsonl`
- Access to Azure ML with the Phi-4-mini model in the registry
- Python environment with necessary libraries (which we'll install)

## Setup Instructions

1. **Azure Authentication**: Ensure you're logged in to Azure using `az login --use-device-code` in a terminal
2. **Kernel Selection**: Change the Jupyter kernel to **"Python 3.10 PyTorch and Tensorflow"** using the selector in the top right
3. **Environment File**: Ensure your `local.env` file exists with proper credentials


## Initial Setup

Before we begin, make sure you've completed these steps:

1. **Azure Login**: Run `az login --use-device-code` in a terminal to authenticate with Azure

2. **Kernel Selection**: Select the "Python 3.10 PyTorch and Tensorflow" kernel from the dropdown in the top-right corner. This kernel has most of the dependencies we need pre-installed.

3. **Check Environment**: Ensure your `local.env` file is in the same directory as this notebook

## 1. Install Authentication Packages

Here we install the packages needed to authenticate with Azure services:

- **azure-ai-ml**: The Azure ML SDK for working with Azure Machine Learning

The `-U` flag ensures we get the latest versions of these packages.

In [24]:
# Install required packages for authentication
! pip install azure-ai-ml -U

## 2. Install PyTorch

Here we install PyTorch, which is the deep learning framework we'll use for fine-tuning. This command installs
 
- **torch**: The core PyTorch library for neural networks and tensor operations
- **torchvision**: For computer vision tasks (included as a dependency)
- **torchaudio**: For audio processing tasks (included as a dependency)
    
We're installing from a specific URL (`download.pytorch.org/whl/cu124`) to get a version compatible with CUDA 12.4, which is optimized for modern NVIDIA GPUs. The `-U` flag ensures we get the latest version."

In [3]:
! pip  install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124 -U

Looking in indexes: https://download.pytorch.org/whl/cu124
  Obtaining dependency information for torch from https://download.pytorch.org/whl/cu124/torch-2.6.0%2Bcu124-cp310-cp310-linux_x86_64.whl.metadata
  Obtaining dependency information for torchvision from https://download.pytorch.org/whl/cu124/torchvision-0.21.0%2Bcu124-cp310-cp310-linux_x86_64.whl.metadata
  Obtaining dependency information for torchaudio from https://download.pytorch.org/whl/cu124/torchaudio-2.6.0%2Bcu124-cp310-cp310-linux_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 768.4/768.4 MB 1.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 119.1 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 92.7 MB/s eta 0:00:00:00:01
Using cached https://download.pytorch.org/whl/cu124/torch-2.6.0%2Bcu124-cp310-cp310-linux_x86_64.whl (768.4 MB)
Using cached https://download.pytorch.org/whl/cu124/torchvision-0.21.0%2Bcu124-cp310-cp310-linux_x86_64.whl 

## 3. Install Microsoft Olive

Now we install Microsoft Olive, an open-source model optimization toolkit that will be the main tool for our fine-tuning and optimization process. The `[auto-opt]` option includes additional dependencies for automatic optimization.

Olive provides:
- Model fine-tuning capabilities
- ONNX conversion tools
- Quantization for model compression
- Performance optimization for various hardware targets

This powerful tool will help us efficiently fine-tune our model and prepare it for deployment on resource-constrained devices.

In [4]:
! pip install olive-ai[auto-opt] -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 688.8/688.8 kB 11.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 689.4/689.4 kB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 961.5/961.5 kB 64.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 429.3/429.3 kB 43.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 78.0 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 12.5 MB/s eta 0:00:00


## 4. Verify Olive Installation

We'll now check the installed version of Olive to ensure it installed correctly. This command shows:
- The package name
- The installed version number
- Where the package is installed
- The package's dependencies

Confirming the version is important as different versions of Olive may have different features or requirements.

In [5]:
! pip show olive-ai

Name: olive-ai
Version: 0.9.1
Summary: Olive: Simplify ML Model Finetuning, Conversion, Quantization, and Optimization for CPUs, GPUs and NPUs.
Home-page: https://microsoft.github.io/Olive/
Author: Microsoft Corporation
Author-email: olivedevteam@microsoft.com
License: MIT License
Location: /anaconda/envs/azureml_py38/lib/python3.10/site-packages
Requires: numpy, onnx, onnxscript, optuna, pandas, pydantic, pyyaml, torch, torchmetrics, transformers
Required-by: 


## 5. Install ONNX Runtime GenAI

Next, we install ONNX Runtime GenAI, a specialized version of ONNX Runtime designed specifically for generative AI models. This package will allow us to:

- Run our optimized model efficiently
- Leverage specialized optimizations for transformer models
- Access adapter-based fine-tuning capabilities

We're installing version 0.7.1 with the `--pre` flag because it's a pre-release version with features we need for our work. Later notebooks will use this to run inference with our optimized model.

In [6]:
! pip install onnxruntime-genai==0.7.1 --pre

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 18.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 76.3 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: onnxruntime
    Found existing installation: onnxruntime 1.17.3
    Uninstalling onnxruntime-1.17.3:
      Successfully uninstalled onnxruntime-1.17.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
azureml-training-tabular 1.60.0 requires onnxruntime~=1.17.3, but you have onnxruntime 1.22.0 which is incompatible.
azureml-training-tabular 1.60.0 requires psutil<5.9.4,>=5.2.2, but you have psutil 6.1.1 which is incompatible.
azureml-training-tabular 1.60.0 requires scipy<1.11.0,>=1.0.0, but you have scipy 1.11.0 which is incompatible.
azureml-train-automl-runtime 1.60.0 requires onnxruntime~=1.17.3, but you have onnxruntime 1.22.0 which is incompatible.
azureml-a

In [7]:
! pip install onnxruntime==1.21.1 -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 73.0 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: onnxruntime
    Found existing installation: onnxruntime 1.22.0
    Uninstalling onnxruntime-1.22.0:
      Successfully uninstalled onnxruntime-1.22.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
azureml-training-tabular 1.60.0 requires onnxruntime~=1.17.3, but you have onnxruntime 1.21.1 which is incompatible.
azureml-training-tabular 1.60.0 requires psutil<5.9.4,>=5.2.2, but you have psutil 6.1.1 which is incompatible.
azureml-training-tabular 1.60.0 requires scipy<1.11.0,>=1.0.0, but you have scipy 1.11.0 which is incompatible.
azureml-train-automl-runtime 1.60.0 requires onnxruntime~=1.17.3, but you have onnxruntime 1.21.1 which is incompatible.
azureml-automl-runtime 1.60.0 requires onnxruntime~=1.17.3, but you have onnxruntime 1.21.1 which

## 6. Package Management

The next few cells handle package management to avoid conflicts. We're:

1. **Uninstalling onnxruntime-gpu** to avoid conflicts with the regular onnxruntime package
2. **Installing regular onnxruntime** for CPU-based inference
3. **Installing additional dependencies** including:
   - bitsandbytes: For efficient quantization
   - transformers: For working with transformer models
   - peft: For parameter-efficient fine-tuning (LoRA)
   - accelerate: For optimized training

These packages will ensure our environment is properly set up for fine-tuning and optimization.

In [8]:
pip uninstall onnxruntime-gpu --yes

Note: you may need to restart the kernel to use updated packages.


In [9]:
! pip install onnxruntime

In [10]:
! pip install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 11.6 MB/s eta 0:00:0000:0100:01


In [11]:
! pip install transformers==4.49.0 -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 55.6 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 121.6 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.30.1
    Uninstalling transformers-4.30.1:
      Successfully uninstalled transformers-4.30.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
azureml-automl-dnn-nlp 1.60.0 requires torch==2.2.2, but you have torch 2.6.0+cu124 which is incompatible.
azureml-automl-dnn-nlp 1.60.0 requires transformers[sentencepiece,torch]<=4.48.0, but you have transformers 4.49.0 which is incompatible.


In [12]:
! pip install azure-ai-ml -U  

In [13]:
! pip install marshmallow==3.23.2 -U   

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.4 MB/s eta 0:00:00
  Attempting uninstall: marshmallow
    Found existing installation: marshmallow 3.26.1
    Uninstalling marshmallow-3.26.1:
      Successfully uninstalled marshmallow-3.26.1


In [14]:
! pip install tf-keras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 22.2 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 80.4 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
  Attempting uninstall: MarkupSafe
    Found existing installation: MarkupSafe 2.0.1
    Uninstalling MarkupSafe-2.0.1:
      Successfully uninstalled MarkupSafe-2.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas-ml 0.6.1 requires enum34, which is not installed.
thinc 8.2.5 requires numpy<2.0.0,>=1.19.0; python_version >= "3.9", but you have numpy 2.1.3 which is incompatible.
scipy 1.11.0 requires numpy<1.28.0,>=1.21.6, but you have numpy 2.1.3 which is incompatible.
scikit-image 0.25.0 requires pillow>=10.1, but you have pillo

In [15]:
! pip install numpy==1.23.5 -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 70.2 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 2.1.3
    Uninstalling numpy-2.1.3:
      Successfully uninstalled numpy-2.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas-ml 0.6.1 requires enum34, which is not installed.
tensorflow 2.19.0 requires numpy<2.2.0,>=1.26.0, but you have numpy 1.23.5 which is incompatible.
scikit-image 0.25.0 requires numpy>=1.24, but you have numpy 1.23.5 which is incompatible.
scikit-image 0.25.0 requires pillow>=10.1, but you have pillow 9.2.0 which is incompatible.
scikit-image 0.25.0 requires scipy>=1.11.2, but you have scipy 1.11.0 which is incompatible.
responsibleai 0.36.0 requires networkx<=2.5, but you have networkx 3.4 which is incompatible.
dask-sql 2024.5.0 requires dask[dataframe]>=2024.4.1, but you

In [16]:
! pip install peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.1/411.1 kB 8.4 MB/s eta 0:00:00a 0:00:01


In [17]:
! pip list

Package                                    Version
------------------------------------------ --------------
absl-py                                    2.2.2
accelerate                                 0.34.2
adal                                       1.2.7
adlfs                                      2024.12.0
aiofiles                                   22.1.0
aiohappyeyeballs                           2.6.1
aiohttp                                    3.11.16
aiohttp-cors                               0.8.1
aiosignal                                  1.3.2
aiosqlite                                  0.21.0
alembic                                    1.15.2
annotated-types                            0.7.0
ansicolors                                 1.1.8
antlr4-python3-runtime                     4.13.2
anyio                                      4.9.0
applicationinsights                        0.11.10
arch                                       5.6.0
argcomplete                                3.

In [18]:
! pip install peft

## 7. Fine-tune with Low-Rank Adaptation (LoRA)

This is the core command that fine-tunes our student model. We're using Microsoft Olive's fine-tuning capabilities with LoRA (Low-Rank Adaptation), a parameter-efficient fine-tuning method. Here's what each parameter does:

- **`--method lora`**: Use Low-Rank Adaptation, which adds small trainable matrices to key layers instead of updating all weights

- **`--model_name_or_path`**: The base model to fine-tune (Phi-4-mini-instruct from Azure ML registry)

- **`--trust_remote_code`**: Allow execution of code from the remote model repository

- **`--data_name json`**: The format of our training data (JSON)

- **`--data_files`**: Path to our training data generated from the teacher model

- **`--text_template`**: Template for formatting inputs and outputs during training

- **`--max_steps 100`**: Only train for 100 steps (for speed, in production you'd use more)

- **`--output_path`**: Where to save the fine-tuned model and adapter

- **`--target_modules`**: Which layers to apply LoRA to (attention and feed-forward layers)

- **`--log_level 1`**: Set verbosity of logging

This process will take several minutes to complete. It creates a LoRA adapter that captures the knowledge our model learned from the teacher without modifying the base model weights.


In [19]:
! olive finetune \
    --method lora \
    --model_name_or_path  azureml://registries/azureml/models/Phi-4-mini-instruct/versions/1 \
    --trust_remote_code \
    --data_name json \
    --data_files ./data/train_data.jsonl \
    --text_template "<|user|>{Question}<|end|><|assistant|>{Answer}<|end|>" \
    --max_steps 100 \
    --output_path models/phi-4-mini/ft \
    --target_modules "q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj" \
    --log_level 1

Loading HuggingFace model from {'type': 'azureml_registry_model', 'registry_name': 'azureml', 'name': 'Phi-4-mini-instruct', 'version': '1'}
[2025-05-20 20:36:36,950] [INFO] [run.py:142:run_engine] Running workflow default_workflow
[2025-05-20 20:36:36,956] [INFO] [cache.py:138:__init__] Using cache directory: /afh/projects/cvi-lab329-h-3-d0ca370a-6510-40b5-b1dc-98d97b208684/shared/Users/cedricvidal/.olive-cache/default_workflow
[2025-05-20 20:36:37,059] [INFO] [accelerator_creator.py:217:create_accelerators] Running workflow on accelerator specs: gpu-cuda
Subtype value SAS has no mapping, use base class DataReferenceCredentialDto.

Your file exceeds 100 MB. If you experience low speeds, latency, or broken connections, we recommend using the AzCopyv10 tool for this file transfer.

Example: azcopy copy 'https://amlwlrt4use01.blob.core.windows.net/azureml-f0d43bc8-5206-5f61-b419-b5ca8a19bb42/mlflow_model_folder' '/afh/projects/cvi-lab329-h-3-d0ca370a-6510-40b5-b1dc-98d97b208684/shared/Us

## 8. Reinstall ONNX Runtime GenAI

Here we reinstall ONNX Runtime GenAI to ensure we have the correct version after all our package management. This is a precautionary step to make sure we have the version (0.7.1) needed for our model optimization in the next step.

In [20]:
! pip install onnxruntime-genai==0.7.1 --pre

In [21]:
! pip install onnxruntime==1.21.1 -U

In [22]:
 ! pip install protobuf==3.20.3 -U 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.8 MB/s eta 0:00:0000:010:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.6
    Uninstalling protobuf-4.25.6:
      Successfully uninstalled protobuf-4.25.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.19.0 requires numpy<2.2.0,>=1.26.0, but you have numpy 1.23.5 which is incompatible.
mlflow-skinny 2.21.3 requires packaging<25, but you have packaging 25.0 which is incompatible.
azureml-training-tabular 1.60.0 requires onnxruntime~=1.17.3, but you have onnxruntime 1.21.1 which is incompatible.
azureml-training-tabular 1.60.0 requires psutil<5.9.4,>=5.2.2, but you have psutil 6.1.1 which is incompatible.
azureml-training-tabular 1.60.0 requires scipy<1.11.0,>=1.0.0, but you have scipy 1.11.0 which is incompatible.
azureml-train-automl-runtime 1.60.0 requ

## 9. Optimize and Quantize the Model

This command uses Microsoft Olive's auto-optimization capabilities to convert our fine-tuned model to ONNX format and apply int4 quantization. Here's what each parameter does:

- **`--model_name_or_path`**: The base model from Azure ML registry

- **`--adapter_path`**: Path to our LoRA adapter created in the previous step

- **`--device cpu`**: Target CPU for optimization (you could also use cuda for GPU)

- **`--provider CPUExecutionProvider`**: Use the CPU execution provider for ONNX Runtime

- **`--use_model_builder`**: Use Olive's model builder for optimized conversion

- **`--precision int4`**: Apply int4 quantization, which reduces model size by up to 75% compared to FP16

- **`--output_path`**: Where to save the optimized model

- **`--log_level 1`**: Set verbosity of logging

The optimization process:
1. Merges the base model with our LoRA adapter
2. Converts to ONNX format, which is more efficient for inference
3. Applies int4 quantization to dramatically reduce model size
4. Optimizes the model for CPU inference

This process will take several minutes to complete. The result will be a much smaller, more efficient model that can run on devices with limited resources while maintaining most of the accuracy.


In [23]:
! olive auto-opt \
    --model_name_or_path  azureml://registries/azureml/models/Phi-4-mini-instruct/versions/1 \
    --adapter_path models/phi-4-mini/ft/adapter \
    --device cpu \
    --provider CPUExecutionProvider \
    --use_model_builder \
    --precision int4 \
    --output_path models/phi-4-mini/onnx \
    --log_level 1

Loading HuggingFace model from {'type': 'azureml_registry_model', 'registry_name': 'azureml', 'name': 'Phi-4-mini-instruct', 'version': '1'}
[2025-05-20 20:44:12,127] [INFO] [run.py:142:run_engine] Running workflow default_workflow
[2025-05-20 20:44:12,134] [INFO] [cache.py:138:__init__] Using cache directory: /afh/projects/cvi-lab329-h-3-d0ca370a-6510-40b5-b1dc-98d97b208684/shared/Users/cedricvidal/.olive-cache/default_workflow
[2025-05-20 20:44:12,170] [INFO] [accelerator_creator.py:217:create_accelerators] Running workflow on accelerator specs: cpu-cpu
[2025-05-20 20:44:12,173] [INFO] [engine.py:223:run] Running Olive on accelerator: cpu-cpu
[2025-05-20 20:44:12,173] [INFO] [engine.py:864:_create_system] Creating target system ...
[2025-05-20 20:44:12,173] [INFO] [engine.py:867:_create_system] Target system created in 0.000080 seconds
[2025-05-20 20:44:12,173] [INFO] [engine.py:879:_create_system] Creating host system ...
[2025-05-20 20:44:12,173] [INFO] [engine.py:882:_create_syste